# Topic Modeling

In [9]:
# Imports
import warnings
warnings.simplefilter("ignore")

import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

import numpy as np
import pandas as pd

from nltk.corpus import stopwords

import spacy
# nlp = spacy.load('de_core_news_lg')

In [4]:
# Preprocess functions
def keep_nouns(text):
    nouns = ""
    for word in nlp(text):
        if word.pos_ == 'NOUN':
            nouns += " "+word.orth_+" "
    return nouns

def lemmatize(text):
    text = nlp(text)
    return ' '.join([x.lemma_ for x in text])

def preprocess(text):
    return lemmatize(keep_nouns(text))

In [5]:
# Read Data
df = pd.read_csv("Data/clean_thesis.csv", 
                 parse_dates=['Datum'],
                 infer_datetime_format=True
                 )

In [6]:
# Only German Articles
df = df[df.Länder == "Germany, Federal Republic of"].reset_index(drop=True)

In [7]:
# Filter Irrelevant Sources
quellen = pd.read_excel("Data/LexisNexis Quellen.xlsx")
df = df[df['Quelle'].isin(quellen[quellen.relevant == 1.0].Quelle)]

In [11]:
# Drop All Outside Daterange
# df['Datum'] = df.Datum.apply(lambda x: x.date())
start = datetime.datetime.strptime('2015-01-01', "%Y-%m-%d").date()
end = datetime.datetime.strptime('2020-08-31', "%Y-%m-%d").date()
drange = (df['Datum'] >= start) & (df['Datum'] <= end)
df = df.loc[drange]
df['Datum'] = df.Datum.apply(lambda x: pd.to_datetime(x))

In [12]:
# Sample
sample = df.sample(10000).reset_index(drop=True)

In [13]:
# TF VECTORIZATION
tf_vectorizer = CountVectorizer(preprocessor=preprocess)
dtm_tf = tf_vectorizer.fit_transform(sample.Text)
# print(dtm_tf.shape)

In [14]:
# TF-IDF VECTORIZATION
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())
dtm_tfidf = tfidf_vectorizer.fit_transform(sample.Text)
# print(dtm_tfidf.shape)

In [17]:
# Perform LDA

# for TF DTM
lda_tf = LDA(n_components=10, random_state=0)
lda_tf.fit(dtm_tf)
# for TFIDF DTM
lda_tfidf = LDA(n_components=10, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=10, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [19]:
# Visualize LDA Result
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.112709  0.096106       1        1  18.639896
9      0.149266 -0.025921       2        1  16.306873
8     -0.002725  0.048732       3        1  12.829668
0      0.180734  0.037479       4        1  11.284287
7      0.078036  0.037109       5        1  11.240258
6     -0.109481 -0.074076       6        1   9.343184
2     -0.060186 -0.082279       7        1   7.081401
3     -0.224429  0.195487       8        1   6.992898
5     -0.057365 -0.127596       9        1   4.570754
1     -0.066561 -0.105043      10        1   1.710781, topic_info=           Term          Freq         Total Category  logprob  loglift
122369      Uhr   7875.000000   7875.000000  Default  30.0000  30.0000
33854      Euro  12573.000000  12573.000000  Default  29.0000  29.0000
118452  Telefon   2303.000000   2303.000000  Default  28.0000  28.0000
93903   Prozent   7034.000000   7034.000000  Default  27.0000  27.0000
43814   Gebäude   3878.000000   3878.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
77029     Markt     18.954441   1369.601711  Topic10  -7.0552  -0.2120
69374     Kunde     18.824287   1909.659295  Topic10  -7.0621  -0.5513
34187   Experte     17.453222    954.145433  Topic10  -7.1378   0.0669
30306      Ende     17.744889   2726.332434  Topic10  -7.1212  -0.9664
15116   Betrieb     17.284097   2119.486120  Topic10  -7.1475  -0.7409

[888 rows x 6 columns], token_table=        Topic      Freq                            Term
term                                                   
81          6  0.953843                      112Polizei
128         6  0.902467             133333Notfallpraxis
233         1  0.994838  197APOTHEKENNOTDIENSTNotdienst
535         1  0.991168                   33Selbsthilfe
671         1  0.985414                       444Kinder
...       ...       ...                             ...
146557     10  0.009423                             www
147227      1  0.243140                     Ölheizungen
147227      2  0.008684                     Ölheizungen
147227      4  0.746788                     Ölheizungen
147349      9  0.983941                       Ölverlust

[3163 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 10, 9, 1, 8, 7, 3, 4, 6, 2])